In [ ]:
import xarray as xr
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import skill_metrics as sm
# import proplot as pplt

## 读入数据

2022.01.08

- 因为xarray会将读入nc文件含有units = days 的数据识别为日期，decode_times 需要关闭，从而读入dtype为float64的 天数，而非一个时间dtype

In [ ]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res/"
filename_obs  = "mask_sel_CN05.1_Tm_1961_2018_daily_025x025.nc"
filename_vr     = "mask_mean_t2m_98-17_VR.nc"
filename_rcm    = "mask_mean_t2m_98-17_RCM.nc"

# 平均气温
ds_or = {}
ds_or['obs'] = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['obs'] = ds_or['obs']['tm']#[:,  :, :]
var['vr'] = ds_or['vr']['t2m'] - 273.15
var['rcm'] = ds_or['rcm']['t2m'] - 273.15

# var['obs'] = var['obs'].reset_coords(names = 'lev', drop = True) # 去除掉obs中多余的lev coords

# change coords
var_list = ['obs', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var[i].coords.keys(), var['obs'].coords.keys()))
#     # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var[i] = var[i].rename(rename_dict)
    var[i]._coords = var['obs']._coords
    var[i] = var[i].rename(i)

In [ ]:
ds_in_extreme = {}
for mod_name in ['obs','vr', 'rcm']:
    ds_in_extreme[mod_name] = {}
    ds_in_extreme[mod_name]['am'] = xr.open_dataset("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/t2m/extreme/extreme_"+mod_name+"_4-5.nc", decode_times = False)
    ds_in_extreme[mod_name]['jja'] = xr.open_dataset("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/t2m/extreme/extreme_"+mod_name+"_6-8.nc", decode_times = False)
    ds_in_extreme[mod_name]['amjja'] = xr.open_dataset("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/t2m/extreme/heatwave_"+mod_name+"_4-8.nc", decode_times = False)
vars_names = list(ds_in_extreme[mod_name]['am'].variables)[3:]
vars_names

## 计算部分

### 平均态计算

In [ ]:
indices_mean = {}

time_idx = {}
time_idx['am'] = var['obs'].time.dt.month.isin([4,5])
time_idx['jja'] = var['obs'].time.dt.month.isin([6,7,8])

for mod_name in ['obs', 'vr', 'rcm']:
    indices_mean[mod_name] = {}
    for iseason in ['am','jja','amjja']:
        indices_mean[mod_name][iseason] = {}
        if (iseason in ['am','jja']):
            # 平均态
            var_selmonth = var[mod_name].isel(time = time_idx[iseason]) # 选取月份的mod变量，后续通过它计算指标
            indices_mean[mod_name][iseason]['mean'] =  var_selmonth.mean(dim = "time")
            # 计算出的极端指数
            for index_name in vars_names:
                indices_mean[mod_name][iseason][index_name] = ds_in_extreme[mod_name][iseason][index_name].mean(dim = 'time')
        if (iseason in ['amjja']):
            for index_name in ['heat_wave_total_length','heat_wave_max_length']:
                indices_mean[mod_name][iseason][index_name] = ds_in_extreme[mod_name][iseason][index_name].mean(dim = 'time')

### 泰勒图诸要素计算

In [ ]:
def nested_dict():
    from collections import defaultdict
    return defaultdict(nested_dict)
# 进行mask，并且整理成1D数组，得到1D-pattern，用于后续的计算
# 计算指数的平均值

indices_mean_1d = nested_dict()
for mod_name in ['obs', 'vr', 'rcm']:
    for iseason in ['am','jja','amjja']:
        indices_names = list(indices_mean['obs'][iseason].keys())
        for count_way in indices_names:
            # 获取1D的平均态 并以RCM为基础进行NAN的MASK
            rcm_not_nan = ( ~np.isnan(indices_mean['rcm'][iseason][count_way])) &\
                 (~np.isnan(indices_mean['obs'][iseason][count_way])) &\
                 (~np.isnan(indices_mean['vr'][iseason][count_way]) )
            var_1d_temp = xr.where(rcm_not_nan, indices_mean[mod_name][iseason][count_way], np.nan).values.ravel()
            indices_mean_1d[mod_name][iseason][count_way] = var_1d_temp[~np.isnan(var_1d_temp)]
# shape check
# for iseason in ['am','jja']:
#     for count_way in indices_names:
#         print("shape check!\n OBS     VR     RCM")
#         print(str(indices_mean_1d['obs'][iseason][count_way].shape) +' '\
#         +str(indices_mean_1d['vr'][iseason][count_way].shape) +' ' \
#         +str(indices_mean_1d['rcm'][iseason][count_way].shape))

In [ ]:
# 计算泰勒图诸要素 到字典taylor_ts中

# 进行mask，并且整理成1D数组，得到1D-pattern，用于后续的计算
# 计算指数的平均值
taylor_space = nested_dict()

for mod_name in ['obs', 'vr', 'rcm']:
    for iseason in ['am','jja','amjja']:
        indices_names = list(indices_mean['obs'][iseason].keys())
        for count_way in indices_names:
            taylor_space[mod_name][iseason][count_way] = {} 
            temp_obs = indices_mean_1d['obs'][iseason][count_way]
            temp_mod = indices_mean_1d[mod_name][iseason][count_way]
            # remove nan
            temp_obs = temp_obs[~np.isnan(temp_obs)]
            temp_mod = temp_mod[~np.isnan(temp_mod)]
            # taylor count
            taylor_space[mod_name][iseason][count_way] = sm.taylor_statistics(temp_mod, temp_obs)
            # normalized 注意先后顺序，先归一化CRMSD，再SDEV，否则用归一化后的SDEV计算RMSD会出现问题
            taylor_space[mod_name][iseason][count_way]['crmsd'] = taylor_space[mod_name][iseason][count_way]['crmsd'] / taylor_space[mod_name][iseason][count_way]['sdev'][0]
            taylor_space[mod_name][iseason][count_way]['sdev'] = taylor_space[mod_name][iseason][count_way]['sdev'] / taylor_space[mod_name][iseason][count_way]['sdev'][0]

## 绘图部分

逐个点的绘制泰勒图，单独为每个点的属性设置相应的特征

### 检查数据-制表

In [ ]:
table_taylor_metrics[i].values[7]

In [ ]:
table_taylor_metrics = []
for model in ['vr', 'rcm']:
    for season in ['am','jja','amjja']:
        table_temp = pd.DataFrame.from_dict( taylor_space[model][season] ) 
        table_temp['model']  = model
        table_temp['season'] = season
        table_taylor_metrics.append(table_temp)

table_taylor_metrics = pd.concat(table_taylor_metrics)
# table_taylor_metrics
# table_taylor_metrics.loc['crmse']

indices_names = list(indices_mean['obs']['am'].keys())
for i in indices_names:

    real_values_list = []
    for single_value in table_taylor_metrics[i].values:
        if ( (type(single_value) != float) ):
            real_single_values = single_value[1]
        else:
            real_single_values = single_value
        real_values_list.append(real_single_values)
    real_values = np.array(real_values_list)

    table_taylor_metrics[i] = real_values
    
# table_taylor_metrics.loc['sdev']
table_taylor_metrics.to_csv("./output_table/t2m_SpatialTaylor_Indices_2022.01.12.csv")
table_taylor_metrics.loc['sdev']

In [ ]:
indices_names

In [ ]:

# fig, axs = pplt.subplots(ncols=1 ,nrows=1)
fig = plt.figure(figsize=(6, 6), dpi = 600)

first_plot = False # 用作后续overlay设置

# 创建风格的dict,之后传入每个单独的点来进行绘制
# # Define markers
# kind = ['+','o','x','s','d','^','v','p','h','*']
# colorm = ['b','r','g','c','m','y','k']
imodiseason_color = {'vr am':"darkorange", 'vr jja':"red",'vr amjja':"red",'rcm am':'cyan', "rcm jja":"blue", "rcm amjja":"blue"}
# iregion_style_color = {"NE":"m", "NC":"y", "YZ":"c", "SC":"b",'SW':"g",'NWC':"tab:gray"}
# iseason_style_marker = {'am':'.',"jja":'v'}

count_way_marker = {'mean':"o","daily_temperature_range":"X","daily_temperature_range_variability":"d","heat_wave_total_length":"^","heat_wave_max_length":"v"}

count_way_marker_realnames = {'mean':"mean","daily_temperature_range":"range","daily_temperature_range_variability":"range variability",\
    "heat_wave_total_length":"heat wave total length","heat_wave_max_length":"heat wave max length"}


for imod in ['vr','rcm']:
    for iseason in ['am','jja','amjja']:
        indices_names = list(indices_mean['obs'][iseason].keys())
        for count_way in indices_names:
            if ( (count_way in ['heat_wave_total_length','heat_wave_max_length']) & (iseason in ['am','jja']) ):
                break
            # 获取每个点的泰勒图诸要素
            sdev_plot = taylor_space[imod][iseason][count_way]['sdev']
            crmsd_plot = taylor_space[imod][iseason][count_way]['crmsd']
            ccoef_plot = taylor_space[imod][iseason][count_way]['ccoef']

            sm.taylor_diagram(sdev_plot,crmsd_plot,ccoef_plot, 
                #----- mark setting -----
                MarkerDisplayed = 'marker',
                # markerLabel = label2, 
                # numberPanels = 1,
                # markerLabel = ['obs','vr','rcm'],
                # markerLabel = ['obs', imod.upper()],
                # markerLabelColor = imod_style_color[imod],
                markerlegend = "off",
                markerColor = imodiseason_color[imod + " " + iseason],
                # markerColor = 'k',
                markerSize = 6,
                alpha = .5, 
                # -----RNSD-----
                # tickRMS = np.arange(0,1.8,0.2),
                tickRMS = np.arange(0,2.0,0.2),
                tickRMSangle = 70, 
                colRMS = 'g', styleRMS = ':', widthRMS = 1.5,
                # tickCOR = lsCOR,
                showlabelsRMS = "on",
                titleRMS = "off",
                rmslabelformat = ':.1f',
                # -----COR-----
                colCOR = "k", 
                widthCOR = 1.0,
                # -----STD------
                tickSTD = np.arange(0,2.8,.2), axismax = 2.6, 
                showlabelsSTD = 'on',
                # markersymbol = iseason_style_marker[iseason],
                # markersymbol = combine_mod_season_marker[imod + " " + iseason],
                markersymbol = count_way_marker[count_way],
                # -----marker reference-----
                styleOBS = '-', 
                colOBS = 'k',
                markerobs = 'o', 
                overlay = first_plot,
                titleOBS = 'observation',
                # ------ check -----
                checkStats = True,
            )

            first_plot = True


# legend设置，颜色以及型号
# custom legend
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
legend_elements = []
for i in imodiseason_color.items():
    legend_elements.append( Patch(facecolor=i[1],label=i[0].upper()))
for i in count_way_marker.items():
    legend_elements.append( Line2D([0], [0], marker=i[1], color='w', label=count_way_marker_realnames[i[0]],linestyle=None,
                          markerfacecolor='k', markersize=6)) 

fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.95,0.95),prop={'size': 6}, ncol=1)

fig.suptitle("surface air temperature")
fig.text(0.025,.95,"(b)", fontsize = 16, fontweight = "bold", color = 'k')

#saving pics
# plt.savefig("./output_pic/t2m_SpatialTaylor_Indices_2022.01.13.png",dpi=600, facecolor = 'white')